In [15]:
import pandas as pd
import csv
import pandas_profiling as pp
import matplotlib.pyplot as plt
import numpy as np

In [131]:
df = pd.read_csv('/Users/PatrickChen/Desktop/dsi-atx-1-projects-pchen2890/CAPSTONE/datafiniti.csv', encoding='UTF-8')
# drop unnessary columns
df = df.drop(["type", "rules","people","listingType","key","fees","encoding",
              "dateUpdated","country","taxID","unitName","address","availablities","brokers",
              "deposit","garageSize","hours","isAvailable","images","languages","leasingTerm",
              "lotSize","managedBy","mlsNumber","nearbySchools","neighborhood","numFields","numFloor",
              "numUnit","parking","paymentTypes","petPolicy","phone","possession","postalCode",
              "propertyTax","schoolDistrict","secondary","sizes","ids","reviews","rules","people",
              "type","leasingTerm","hours"],axis =1)

In [132]:
#Only get austin texas
df = df[df.city=="Austin"]
df = df[df.province=='TX']

In [133]:
# pp.ProfileReport(df)

In [134]:
# covert prices to floats
import ast
df = df[df.prices.notnull()]
df.prices = df.prices.astype(str)
df.prices = df.prices.map(lambda x: ast.literal_eval(x)[0]["price"])
df.prices = df.prices.str.strip("[]")
df.prices = df.prices.str.strip("USD")
df.prices = df.prices.astype(float)

In [136]:
# Change dates
df.dateAdded = pd.to_datetime(df.dateAdded)
df.dateAdded = [d.date() for d in df.dateAdded]

In [137]:
df.columns

Index([u'city', u'dateAdded', u'descriptions', u'features', u'lat', u'long',
       u'name', u'numBathroom', u'numBedroom', u'numBed', u'numPeople',
       u'propertyType', u'prices', u'province', u'sourceURLs'],
      dtype='object')

In [138]:
# get only columns needed
df = df[["prices","dateAdded","lat","long","propertyType",'numBathroom',"numBedroom","numBed","numPeople"]]

In [140]:
# get zipcodes from coordinates
from tqdm import tqdm

import geocoder
zipcodes = []
house_num = []
street_name = []
for x,y in tqdm(zip(df['lat'],df['long'])):
    g = geocoder.google([x,y], method='reverse')
    house_num.append(g.housenumber)
    street_name.append(g.street_long)
    zipcodes.append(g.postal)

df["house_num"] = house_num
df["street_name"] = street_name
df['zipcodes'] = zipcodes

100%|██████████| 1965/1965 [09:15<00:00,  3.76it/s]


In [141]:
df.head()

,prices,dateAdded,lat,long,propertyType,numBathroom,numBedroom,numBed,numPeople,house_num,street_name,zipcodes
0,107.0,2015-10-28,30.261685,-97.707917,House,1.0,1.0,1.0,2.0,2816,Prado Street,78702
1,40.0,2016-08-05,30.435890,-97.729738,Condominium,1.0,1.0,1.0,3.0,13021,Legendary Drive,78727
2,59.0,2016-03-30,30.406389,-97.717191,Apartment,1.0,1.0,1.0,2.0,11902,Burnet Road,78758
3,59.0,2015-11-05,30.439717,-97.753116,House,1.0,1.0,1.0,2.0,6508,McNeil Drive,78729
4,140.0,2016-04-04,30.338714,-97.857038,Other,1.0,0.0,1.0,2.0,1801,Corto Lane,78733


In [143]:
df.isnull().sum(axis=0)

prices            0
dateAdded         0
lat               0
long              0
propertyType    193
numBathroom      13
numBedroom        2
numBed          200
numPeople         0
house_num        67
street_name       1
zipcodes          0
dtype: int64

In [146]:
# Remove nulls
df2 = df[df.street_name.notnull()]
df2 = df2[df.house_num.notnull()]

In [147]:
# remove those with 1777-1888
df2.house_num = df2.house_num.apply(lambda x: x.split("-")[0])

In [149]:
df2.propertyType.value_counts()

House            889
Apartment        650
Condominium       65
Loft              22
Townhouse         19
Other             16
Bungalow          12
Camper/RV         10
Bed Breakfast     10
Guesthouse         4
Cabin              4
Boat               3
Villa              2
Tent               1
Lighthouse         1
Name: propertyType, dtype: int64

In [190]:
# Only get property type of house, apartment, loft, townhomes
df2 = df2[(df2.propertyType=="House")|(df2.propertyType=="Apartment")|(df2.propertyType=="loft")|(df2.propertyType=="Townhouse")]

In [191]:
df2.dtypes

prices          float64
dateAdded        object
lat             float64
long            float64
propertyType     object
numBathroom     float64
numBedroom      float64
numBed          float64
numPeople       float64
house_num        object
street_name      object
zipcodes          int64
dtype: object

In [192]:
#remove unicode issue with convert to csv
df2.dateAdded = df2.dateAdded.astype(str)
df2.propertyType = df2.propertyType.astype(str)
df2.house_num = df2.house_num.astype(str)
df2.zipcodes = df2.zipcodes.astype(int)

df2.street_name = df2.street_name.apply(lambda x: str(x.encode('utf-8').decode('ascii', 'ignore')))

In [193]:
df2.to_csv('/Users/PatrickChen/Desktop/dsi-atx-1-projects-pchen2890/CAPSTONE/airbnb_data.csv',encoding='ascii')